In [281]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *
from row_utils import *

path_2004 = "./errors/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [232]:
results = []
for i, fname in enumerate(all_2004):
    print("File: ", i)
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1]  

    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            multiplier = get_multiplier_from_tbl_list(tbl_list)

            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
pd.DataFrame(results).to_csv('errors_df')

File:  0
No HTML tables!
File:  1


/Users/divyas/Documents/G1/sec/utils.py:179: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 179 of the file /Users/divyas/Documents/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)


Found a table
File:  2
No contractual tables!
File:  3
No HTML tables!
File:  4
No HTML tables!
Returning:  []
File:  5
No HTML tables!
File:  6
No contractual tables!
File:  7
No HTML tables!
Returning:  []
File:  8
No HTML tables!
File:  9
No HTML tables!
Unknown error:  list index out of range
File:  10
No HTML tables!
Returning:  []
File:  11
No HTML tables!
Returning:  []
File:  12
No HTML tables!
Unknown error:  list index out of range
File:  13
No contractual tables!
Returning:  []
File:  14
No HTML tables!
File:  15
Found a table
File:  16
No HTML tables!
Returning:  []
File:  17
No HTML tables!
Unknown error:  list index out of range
File:  18
No HTML tables!
File:  19
No HTML tables!
File:  20
No HTML tables!
File:  21
No HTML tables!
Unknown error:  list index out of range
File:  22
No HTML tables!
File:  23
No HTML tables!
File:  24
No HTML tables!
Returning:  []
File:  25
No contractual tables!
Unknown error:  list index out of range
File:  26
No HTML tables!
File:  27
No 

In [301]:
#results_df = pd.DataFrame(results)
results_df = pd.read_csv('scraped_files_df')
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print(len(set(last_check['fname'])), len(set(row_check['fname'])), len(set(none_check['fname'])))

73 18 0


In [302]:
row_check

,Unnamed: 0,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
13,13,-,-,-,-,NaN,1017172,20041231,0000950135-05-001504.txt,1,0,1,-
22,22,-,-,-,-,NaN,1016169,20041231,0001016169-05-000005.txt,1,0,1,-
76,76,-,-,-,-,NaN,895347,20041231,0001193125-05-065291.txt,1,0,1,-
88,88,-,-,-,-,NaN,1166041,20041231,0001104659-05-013957.txt,1000,0,1,-
96,96,-,-,-,-,NaN,871763,20041231,0001193125-05-034776.txt,1000000,0,1,-
101,101,-,-,-,-,NaN,879555,20050331,0001104659-05-032569.txt,1,0,1,-
102,102,-,-,-,-,NaN,1074433,20050930,0001074433-05-000052.txt,1000,0,1,-
117,117,-,-,-,-,NaN,217346,20050101,0001104659-05-008155.txt,1000000,0,1,-
130,130,-,-,-,-,NaN,753281,20041231,0001104659-05-009937.txt,1000,0,1,-
138,138,-,-,-,-,NaN,1030839,20021031,0001144204-05-000880.txt,1,0,1,-


# Examining one file

In [316]:
path_prefix = "./2005/"
test_f = open(path_prefix + "0001144204-05-009084.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()
#lines, tbl_list = scrape_text_table(wholetext)
lines, tbl_list = scrape_text_table(wholetext)
rows = scrape_text_rows(lines, tbl_list)
#scores, tables = scrape_table(wholetext)
#table = scrape_table(wholetext)
#rows = scrape_rows(table)
#lines
rows

CIK:  0000096988 Date:  20041231
['contractual obligations total', '1', 'year years years years']
['commitment for this facility is', '214000', 'through september', '22', '2007', 'which is']
['store fixtures will be essentially unchanged in', '2005', 'as compared with', '2004.']
['the company wrote off accounts receivable and inventories in', '2001', 'as a result of']
ROWS
[['long0term debt obligations', '5057', '46', '34', '4977', '00'], ['capital lease obligations', '96', '38', '58', '00', '00'], ['operating lease obligations', '7427', '2100', '4601', '726', '00']]
total_idx:  -1
['capital lease obligations', '96', '38', '58', '00', '00']


[{'1-3': 34.0,
  '3-5': '4977',
  '<1': 46.0,
  '>5': '00',
  'category': 'long0term debt obligations',
  'total': 5057.0},
 {'1-3': 58.0,
  '3-5': '00',
  '<1': 38.0,
  '>5': '00',
  'category': 'capital lease obligations',
  'total': 96.0},
 {'1-3': 4601.0,
  '3-5': '726',
  '<1': 2100.0,
  '>5': '00',
  'category': 'operating lease obligations',
  'total': 7427.0}]

In [209]:
#table_to_list(table)
table_to_list(tables[np.where(scores == 4)[0][1]])
#scores

IndexError: index 1 is out of bounds for axis 0 with size 1

In [293]:
#lines = chunks[np.argmax(scores)].split('\n')
lines = [x for x in lines if (hasNumbers(x))]
keywords = HDR_KWS + ROW_KWS + ['obligations']
lines = [x for x in lines if (hasKeywords(x, keywords))]
lines = [x.split(' ') for x in lines]
for i, line in enumerate(lines):
    lines[i] = list(filter(lambda a: (a != '') and (a != '$') and (a !='<u>') and (a !='</u>'), line))    
tbl_list = []
tbl_list = combine_text_fields(lines)
scrape_text_rows(lines, tbl_list)

Unknown error:  could not convert string to float: 'by inflation of'


[{'1-3': 2848000.0,
  '3-5': '71000',
  '<1': 1179000.0,
  '>5': '000',
  'category': 'obligations',
  'total': 4098000.0},
 {'1-3': 24293.0,
  '3-5': '0',
  '<1': 24293.0,
  '>5': '0',
  'category': 'obligations',
  'total': 48586.0},
 {'1-3': 0.0,
  '3-5': '0',
  '<1': 3450.0,
  '>5': '0',
  'category': 'purchase obligations',
  'total': 3450.0}]

In [135]:
#lt[1].text.replace('\n', '').replace('\xa0', '')
len(lt)

1